#### **12. Implement batch gradient descent with early stopping for softmax regression without using Scikit-Learn, only NumPy. Use it on a classification task such as the iris dataset.**

In [2]:
import numpy as np

In [3]:
from sklearn.datasets import load_iris
iris = load_iris(as_frame=True)

In [4]:
X = iris.data.values
X.shape # 150 instances with 4 features each

(150, 4)

In [5]:
y = iris.target_names[iris.target]
print(f"y has {y.shape[0]} instances") 

y has 150 instances


In [6]:
# Splitting the data 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [7]:
(X_train[:, 1] - X_train[:, 1].mean()) / X_train[:, 1].std()

array([ 1.2864604 ,  2.43545215, -0.78172474,  0.8268637 ,  0.13746866,
        1.97585545,  1.51625875, -1.47111979, -0.55192639,  0.13746866,
       -0.09232969,  1.2864604 , -0.32212804, -0.09232969, -0.09232969,
        1.05666205, -0.55192639, -0.78172474,  0.8268637 ,  1.05666205,
        1.7460571 , -2.39031318, -0.78172474,  0.8268637 , -0.09232969,
        0.59706535, -0.78172474,  0.8268637 , -0.09232969,  1.7460571 ,
        0.367267  , -0.32212804, -1.24132144,  2.66525049, -0.09232969,
       -0.78172474, -1.93071649, -0.09232969,  0.8268637 , -1.70091814,
        1.97585545, -1.70091814, -0.78172474,  0.59706535,  0.367267  ,
       -1.47111979, -0.09232969,  0.13746866, -0.55192639,  0.8268637 ,
        1.05666205,  0.367267  , -1.24132144,  0.367267  , -0.78172474,
       -1.01152309, -0.09232969, -1.24132144, -0.32212804, -0.09232969,
       -1.93071649,  0.367267  ,  0.13746866,  1.05666205, -0.09232969,
       -1.24132144,  0.8268637 ,  0.8268637 , -0.09232969,  0.59

In [8]:
# Processing the training data
def standard_scale(np_array, col_idxs):
    standardized_columns = []
    for col_idx in col_idxs:
        standardized_columns.append(
            (np_array[:, col_idx] - np_array[:, col_idx].mean()) / np_array[:, col_idx].std()
        )
    return np.column_stack(standardized_columns)


In [9]:
X_train = standard_scale(X_train, [0, 1, 2, 3])
X_train.shape

(112, 4)

In [10]:
# Checking the unique values (categories)
print(f'Unique values in train set: {np.unique(y_train)}')
print(f'Unique values in train set: {np.unique(y_test)}')

# Replacing each value with a numeric value so that we can calculate the gradients
y_train_num = []
for i in range(len(y_train)):
    if y_train[i] == 'setosa':
        y_train_num.append(0)
    elif y_train[i] == 'versicolor':
        y_train_num.append(1)
    else:
        y_train_num.append(2)

# Just testing a few
for i in range(0, 5):
    print(f'Index: {i}')
    print(f'Actual category: {y_train[i]}')
    print(f'Numeric category: {y_train_num[i]}')
    print('*'*50)

y_train_num = np.array(y_train_num).reshape(-1, 1)

Unique values in train set: ['setosa' 'versicolor' 'virginica']
Unique values in train set: ['setosa' 'versicolor' 'virginica']
Index: 0
Actual category: setosa
Numeric category: 0
**************************************************
Index: 1
Actual category: setosa
Numeric category: 0
**************************************************
Index: 2
Actual category: virginica
Numeric category: 2
**************************************************
Index: 3
Actual category: versicolor
Numeric category: 1
**************************************************
Index: 4
Actual category: versicolor
Numeric category: 1
**************************************************


In [11]:
# Performing the same operation for y_test
# Replacing each value with a numeric value so that we can calculate the gradients
y_test_num = []
for i in range(len(y_test)):
    if y_test[i] == 'setosa':
        y_test_num.append(0)
    elif y_test[i] == 'versicolor':
        y_test_num.append(1)
    else:
        y_test_num.append(2)

y_test_num = np.array(y_test_num)

In [12]:
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train_num.shape}")
print('*'*50)
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test_num.shape}")

X_train shape: (112, 4)
y_train shape: (112, 1)
**************************************************
X_test shape: (38, 4)
y_test shape: (38,)


In [24]:
# Each class should have a weight for each feature
# Since we have 3 classes, and 4 features
# We expect a 3 x 4 matrix (each class' weights for each feature is one row)

# So let us randomly initalize the weights
num_classes = len(np.unique(y_train_num))
num_features = X_train.shape[1]

print(f"Since we have {num_classes} classes, and {num_features} features, \
our weight matrix will have shape {num_classes} x {num_features}.")

weight_matrix = np.random.randn(num_classes, num_features)
print(weight_matrix.shape)

# Logit (raw score) of the first instance being in class 0 with random weights
weight_matrix[0, :].T @ X_train[1]

Since we have 3 classes, and 4 features, our weight matrix will have shape 3 x 4.
(3, 4)
[-1.01827123  1.2864604  -1.39338902 -1.3621769 ]


In [38]:
# About to run batch gradient descent
num_epochs = 1000
learning_rate = 0.02
num_instances = X_train.shape[0]

for epoch in range(0, 1):
    print(f"Current epoch: {epoch}")
    
    # Step 1: Calculate the raw score of each instance being in each class
    raw_scores_vector = []
    for idx in range(len(weight_matrix)):
        class_weights_vector = weight_matrix[idx]
        print(f"Weights for class {idx}: {class_weights_vector}")
        class_raw_score = class_weights_vector.T @ X_train[0]
        print(f"Logit (raw score) for class {idx}: {class_raw_score}")
        print('-'*20)

        raw_scores_vector.append(class_raw_score)
    raw_scores_vector = np.array(raw_scores_vector)
    print('\n\n')

    # Step 2: Calculating a probability score for each instance
    sum_raw_scores_exponents = np.exp(raw_scores_vector).sum()
    for idx in range(len(raw_scores_vector)):
        prob = np.exp(raw_scores_vector[idx]) / sum_raw_scores_exponents
        print(f"Probability of class {idx}: {round(prob, 2) * 100}%")
        print('-'*20)


    



Current epoch: 0
Weights for class 0: [0.74805295 0.20302355 0.21513962 0.46284109]
Logit (raw score) for class 0: -1.4307836849935751
--------------------
Weights for class 1: [ 0.56387081 -1.46685203  0.72642599 -2.48901811]
Logit (raw score) for class 1: -0.08293151217984662
--------------------
Weights for class 2: [-0.88442476  0.3672323  -0.04309255 -0.72483252]
Logit (raw score) for class 2: 2.4204088976171265
--------------------



Probability of class 0: 2.0%
--------------------
Probability of class 1: 7.000000000000001%
--------------------
Probability of class 2: 91.0%
--------------------


In [32]:
np.exp(np.array([1, 2, 3])).sum()

30.19287485057736

In [ ]:
# Implementing batch gradient descent
learning_rate = 0.02
num_epochs = 1000
num_instances = X_train.shape[0]

np.random.seed(42)
# Randomly initialize weights for each feature
theta = np.random.randn(X_train.shape[1], 1)
print(f"Initial Weights: {theta}\n")

# 
for epoch in range(num_epochs):
    gradients = 2 / num_instances * X_train.T @ (X_train @ theta - y_train_num)
    theta = theta - learning_rate * gradients

# Final theta
print(f"Final theta: {theta}")

Initial Weights: [[ 0.49671415]
 [-0.1382643 ]
 [ 0.64768854]
 [ 1.52302986]]

Final theta: [[-0.02153836]
 [-0.05733346]
 [ 0.24250165]
 [ 0.53404859]]
